<a href="https://colab.research.google.com/github/kunwarsharik/SharikProjects/blob/master/ANN_without_any_library_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets develop a basic NN with  1 input 1 output and in between 1 hidden layer.<br>
forward propogation.<br>1.activation of neuron. calc weighted sum + bias. do this for all neuron.

In [0]:
from random import seed,random,randrange
from math import exp
from csv import reader

In [2]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv

--2020-02-11 13:03:20--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9301 (9.1K) [text/plain]
Saving to: ‘wheat-seeds.csv’

wheat-seeds.csv     100%[===================>]   9.08K  --.-KB/s    in 0s      

2020-02-11 13:03:21 (128 MB/s) - ‘wheat-seeds.csv’ saved [9301/9301]



In [0]:
def init_net(n_input,n_hidden,n_output):
    net=[]
    h_layer= [{'weights':[random()for i in range(n_input+1)]} for j in range(n_hidden)]   #keep bias also
    net.append(h_layer)
    o_layer=[{'weights':[random() for i in range(n_hidden+1)]} for j in range(n_output)]
    net.append(o_layer)
    return net

In [4]:
seed(1)
network=init_net(2,1,2)
[print(layer) for layer in network]	

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


[None, None]

In [0]:
def activation(weights,inputs):
    # for 1 neuron we have weights connected to it and a bias, here we take weghted sum with input
    # which can be from hidden layer or input layer which is row of our data
    # if bias is last in weights
    act=weights[-1]
    for i in range(len(weights)-1):
        act+=weights[i]*inputs[i]
    return act

In [0]:
#neuron_output=transfer_func(activation), transfer function like sigmoid(logistic),tanh(hyperbolic tangent), rectifier etc.
#we need derivative in back prop step. sigmoid derivative is easy to calc.
def transfer_func(act):
    return 1.0/(1.0 + exp(-act)) 

In [0]:
#for each neuron apply activation. these then become input to next layer
def forward_propogation(net,row):
    inputs=row
    for layer in net:
        new_inputs=[]
        for neuron in layer:
            act= activation(neuron['weights'],inputs)
            neuron['output']=transfer_func(act)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [8]:
row=[1,0,0.5]
output = forward_propogation(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [0]:
#back propogation is calc b/w expected output and output obtained using backpropogation
# if sigmoid function is used, d/dx(f(x))=f(x)*(1-f(x))
def derivative_trasfer_func(output):#slope of neuron's output value
    return output*(1-output)

In [0]:
# first calc error, for hidden neuron next layer ke sabhi se connected h, un connection ke weights
# in sabka weighted error 
def back_propogation_error(net, expected):
    for i in reversed(range(len(net))):
        layer=net[i]#last layer
        errors=[]
        if i!=len(net)-1:
            for j in range(len(layer)):#for each layer
                error=0
                for neuron in net[i+1]:
                    error+=(neuron['weights'][j]*neuron['delta'])#neuron delta is error for this neuron
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] =errors[j] * derivative_trasfer_func(neuron['output'])

In [11]:
[print(layer) for layer in network]
expected=[1,0]
back_propogation_error(network, expected)
print("")
print("")
[print(layer) for layer in network]

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748}]


[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.009408700063707465}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': 0.07529724255099601}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': -0.1445066558817715}]


[None, None]

In [0]:
#training process
#errors are calculated for each neuron using back_prop. we can update weight,bias using them.
#the rate at which we want to remove errors is learning rate. keep small to avoid premature convergence.
def update_weights(net, row, l_rate):
    for i in range(len(net)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in net[i - 1]]
        for neuron in net[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [0]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propogation(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            back_propogation_error(network,expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [0]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

In [15]:
n_inputs = 2
n_outputs = 2
network = init_net(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=6.772
>epoch=1, lrate=0.500, error=5.934
>epoch=2, lrate=0.500, error=5.561
>epoch=3, lrate=0.500, error=5.427
>epoch=4, lrate=0.500, error=5.375
>epoch=5, lrate=0.500, error=5.346
>epoch=6, lrate=0.500, error=5.320
>epoch=7, lrate=0.500, error=5.289
>epoch=8, lrate=0.500, error=5.243
>epoch=9, lrate=0.500, error=5.167
>epoch=10, lrate=0.500, error=5.037
>epoch=11, lrate=0.500, error=4.843
>epoch=12, lrate=0.500, error=4.619
>epoch=13, lrate=0.500, error=4.378
>epoch=14, lrate=0.500, error=4.112
>epoch=15, lrate=0.500, error=3.825
>epoch=16, lrate=0.500, error=3.530
>epoch=17, lrate=0.500, error=3.236
>epoch=18, lrate=0.500, error=2.954
>epoch=19, lrate=0.500, error=2.691
[{'weights': [1.1150736105858396, -1.3744036810415425, -0.5025574372710625], 'output': 0.9483052263131478, 'delta': 0.00894071061629689}, {'weights': [0.7810655718182448, 0.3654092266825157, 0.7344762774557891], 'output': 0.9996681219747121, 'delta': -4.614097675520693e-06}]
[{'weights': [

In [0]:
def predict(net,row):
    outputs=forward_propogation(net,row)
    return outputs.index(max(outputs))

In [0]:
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

In [18]:
for i in range(len(dataset)):
    pred=predict(network,dataset[i])
    print(dataset[i][-1],pred)

0 0
0 0
0 0
0 0
0 0
1 1
1 1
1 1
1 1
1 1


In [19]:
ls

sample_data/  wheat-seeds.csv


In [0]:
def load_csv(filename):
    dataset=[]
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [0]:
seed(1)
filename = 'wheat-seeds.csv'

In [0]:
dataset = load_csv(filename)

In [28]:
dataset

[['15.26', '14.84', '0.871', '5.763', '3.312', '2.221', '5.22', '1'],
 ['14.88', '14.57', '0.8811', '5.554', '3.333', '1.018', '4.956', '1'],
 ['14.29', '14.09', '0.905', '5.291', '3.337', '2.699', '4.825', '1'],
 ['13.84', '13.94', '0.8955', '5.324', '3.379', '2.259', '4.805', '1'],
 ['16.14', '14.99', '0.9034', '5.658', '3.562', '1.355', '5.175', '1'],
 ['14.38', '14.21', '0.8951', '5.386', '3.312', '2.462', '4.956', '1'],
 ['14.69', '14.49', '0.8799', '5.563', '3.259', '3.586', '5.219', '1'],
 ['14.11', '14.1', '0.8911', '5.42', '3.302', '2.70', '5.00', '1'],
 ['16.63', '15.46', '0.8747', '6.053', '3.465', '2.04', '5.877', '1'],
 ['16.44', '15.25', '0.888', '5.884', '3.505', '1.969', '5.533', '1'],
 ['15.26', '14.85', '0.8696', '5.714', '3.242', '4.543', '5.314', '1'],
 ['14.03', '14.16', '0.8796', '5.438', '3.201', '1.717', '5.001', '1'],
 ['13.89', '14.02', '0.888', '5.439', '3.199', '3.986', '4.738', '1'],
 ['13.78', '14.06', '0.8759', '5.479', '3.156', '3.136', '4.872', '1'],
 [

In [0]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [0]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [0]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [0]:
# backpropagation with Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = init_net(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [0]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


In [0]:
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)

In [36]:
dataset[:10]

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, '1'],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, '1'],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, '1'],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, '1'],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, '1'],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, '1'],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, '1'],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, '1'],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, '1'],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, '1']]

In [37]:
str_column_to_int(dataset, len(dataset[0])-1)

{'1': 2, '2': 1, '3': 0}

In [38]:
dataset[:10]

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 2],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 2],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 2],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, 2],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, 2],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, 2],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, 2],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, 2],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, 2],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, 2]]

In [0]:
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5

In [44]:
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

>epoch=0, lrate=0.300, error=187.204
>epoch=1, lrate=0.300, error=118.143
>epoch=2, lrate=0.300, error=118.143
>epoch=3, lrate=0.300, error=118.143
>epoch=4, lrate=0.300, error=118.143
>epoch=5, lrate=0.300, error=118.143
>epoch=6, lrate=0.300, error=118.143
>epoch=7, lrate=0.300, error=118.143
>epoch=8, lrate=0.300, error=118.143
>epoch=9, lrate=0.300, error=118.143
>epoch=10, lrate=0.300, error=118.143
>epoch=11, lrate=0.300, error=118.143
>epoch=12, lrate=0.300, error=118.143
>epoch=13, lrate=0.300, error=118.143
>epoch=14, lrate=0.300, error=118.143
>epoch=15, lrate=0.300, error=118.143
>epoch=16, lrate=0.300, error=118.143
>epoch=17, lrate=0.300, error=118.143
>epoch=18, lrate=0.300, error=118.143
>epoch=19, lrate=0.300, error=118.143
>epoch=20, lrate=0.300, error=118.143
>epoch=21, lrate=0.300, error=118.143
>epoch=22, lrate=0.300, error=118.143
>epoch=23, lrate=0.300, error=118.143
>epoch=24, lrate=0.300, error=118.143
>epoch=25, lrate=0.300, error=118.143
>epoch=26, lrate=0.300

In [45]:
print('Scores: %s' % scores)

Scores: [47.61904761904761]


In [46]:
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Mean Accuracy: 47.619%


In [0]:
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

In [51]:
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

>epoch=0, lrate=0.300, error=152.354
>epoch=1, lrate=0.300, error=112.187
>epoch=2, lrate=0.300, error=107.753
>epoch=3, lrate=0.300, error=99.911
>epoch=4, lrate=0.300, error=88.847
>epoch=5, lrate=0.300, error=77.992
>epoch=6, lrate=0.300, error=69.673
>epoch=7, lrate=0.300, error=63.361
>epoch=8, lrate=0.300, error=58.053
>epoch=9, lrate=0.300, error=53.234
>epoch=10, lrate=0.300, error=48.777
>epoch=11, lrate=0.300, error=44.733
>epoch=12, lrate=0.300, error=41.173
>epoch=13, lrate=0.300, error=38.126
>epoch=14, lrate=0.300, error=35.566
>epoch=15, lrate=0.300, error=33.437
>epoch=16, lrate=0.300, error=31.669
>epoch=17, lrate=0.300, error=30.197
>epoch=18, lrate=0.300, error=28.962
>epoch=19, lrate=0.300, error=27.918
>epoch=20, lrate=0.300, error=27.027
>epoch=21, lrate=0.300, error=26.259
>epoch=22, lrate=0.300, error=25.592
>epoch=23, lrate=0.300, error=25.008
>epoch=24, lrate=0.300, error=24.492
>epoch=25, lrate=0.300, error=24.032
>epoch=26, lrate=0.300, error=23.621
>epoch=2

In [52]:
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [95.23809523809523, 92.85714285714286, 97.61904761904762, 92.85714285714286, 88.09523809523809]
Mean Accuracy: 93.333%
